In [1]:
# Import library
import pandas as pd
import os
from datetime import datetime
from underthesea import text_normalize
from underthesea import classify
from underthesea import sentiment
import re
from deep_translator import GoogleTranslator

# Read file
mergecomment = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "merged", "mergedcomment_20240402_1111.csv"))

In [2]:
# list columns
mergecomment.columns

Index(['link_item', 'data_product_id_list', 'data_product_id', 'name_comment',
       'content_comment', 'product_variant', 'datetime_comment', 'rating'],
      dtype='object')

In [3]:
# Choose the 'content_comment' column
mergecomment = mergecomment[['data_product_id', 'content_comment']]

In [4]:
# 1. Convert to lower case
mergecomment["lower_comment"] = mergecomment["content_comment"].str.lower()
mergecomment.head(10)


,data_product_id,content_comment,lower_comment
0,65994,"Giao lâu, chất lượng + đóng gói oke","giao lâu, chất lượng + đóng gói oke"
1,65994,"thích em này thật sự, mùi thơm thảo mộc, lành ...","thích em này thật sự, mùi thơm thảo mộc, lành ..."
2,65994,Toner này gần như ko mùi Dạng sệt thoa lên da ...,toner này gần như ko mùi dạng sệt thoa lên da ...
3,65994,"Vô thưởng vô phạt, dễ chịu, không mùi Thiết kế...","vô thưởng vô phạt, dễ chịu, không mùi thiết kế..."
4,65994,giao hàng nhanh với cả xài rất tốt í,giao hàng nhanh với cả xài rất tốt í
5,65994,Rất hài lòng,rất hài lòng
6,65994,"toner dưỡng ẩm tốt, mùi thơm dễ chịu","toner dưỡng ẩm tốt, mùi thơm dễ chịu"
7,65994,Sp tốt. Đóng gói cẩn thận. giao hàng trễ,sp tốt. đóng gói cẩn thận. giao hàng trễ
8,65994,"Thực sự là mình ko thích loại này lắm, da mình...","thực sự là mình ko thích loại này lắm, da mình..."
9,65994,da dầu như mình xài êm.thành phần dịu nhẹ,da dầu như mình xài êm.thành phần dịu nhẹ


In [5]:
# 2. Replace word

# Read file abbreviations
abbs_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "abbreviations.csv"))

# Create a dictionary to map abbreviations to meanings
abbreviation_dict = dict(zip(abbs_df['abbreviation'], abbs_df['meaning']))

# Function to decode abbs
def decode_abbreviations(text, abbreviation_dict):
    # Search for abbreviations in the text and replace them 
    # with their corresponding meanings from the dictionary
    for abbreviation, meaning in abbreviation_dict.items():
        text = re.sub(r'\b' + re.escape(abbreviation) + r'\b', meaning, text)
    return text

# Apply the decode_abbreviations function to the 'content_comment' column
mergecomment["decoded_comment"] = mergecomment["lower_comment"].apply(lambda x: decode_abbreviations(x, abbreviation_dict))


In [26]:
# 3. Translate to English
# Function to translate a batch of sentences from Vietnamese to English
def translate_batch_vietnamese_to_english(texts):
    translated_texts = GoogleTranslator(source='vi', target='en').translate_batch(texts)
    return translated_texts

# Specify the range of rows you want to translate
# 0-500
# 500-1000
# 1000-1500
# 1500-2000
# 2000-2500
# 2500-3000
# 3000-3500
# 3500-4000
# 4000-4500
# 4500-5000
# 5000-end
start_index = 5000
end_index = len(mergecomment)

# Assign the translated comments to the DataFrame directly
mergecomment.loc[start_index:end_index-1, 'translated_comment'] = translate_batch_vietnamese_to_english(mergecomment['decoded_comment'][start_index:end_index].tolist())

# Display the DataFrame with translated comments
mergecomment.head(15)


,data_product_id,content_comment,lower_comment,decoded_comment,translated_comment
0,65994,"Giao lâu, chất lượng + đóng gói oke","giao lâu, chất lượng + đóng gói oke","giao lâu, chất lượng + đóng gói oke","Long delivery, quality + packaging ok"
1,65994,"thích em này thật sự, mùi thơm thảo mộc, lành ...","thích em này thật sự, mùi thơm thảo mộc, lành ...","thích em này thật sự, mùi thơm thảo mộc, lành ...","I really like this one, the scent is herbal an..."
2,65994,Toner này gần như ko mùi Dạng sệt thoa lên da ...,toner này gần như ko mùi dạng sệt thoa lên da ...,toner này gần như không mùi dạng sệt thoa lên ...,This toner is almost odorless and has a thick ...
3,65994,"Vô thưởng vô phạt, dễ chịu, không mùi Thiết kế...","vô thưởng vô phạt, dễ chịu, không mùi thiết kế...","vô thưởng vô phạt, dễ chịu, không mùi thiết kế...","innocuous, pleasant, odorless and difficult to..."
4,65994,giao hàng nhanh với cả xài rất tốt í,giao hàng nhanh với cả xài rất tốt í,giao hàng nhanh với cả xài rất tốt í,Fast delivery and very good to use
5,65994,Rất hài lòng,rất hài lòng,rất hài lòng,Very pleased
6,65994,"toner dưỡng ẩm tốt, mùi thơm dễ chịu","toner dưỡng ẩm tốt, mùi thơm dễ chịu","toner dưỡng ẩm tốt, mùi thơm dễ chịu","Good moisturizing toner, pleasant fragrance"
7,65994,Sp tốt. Đóng gói cẩn thận. giao hàng trễ,sp tốt. đóng gói cẩn thận. giao hàng trễ,sản phẩm tốt. đóng gói cẩn thận. giao hàng trễ,good product. careful packing. late delivery
8,65994,"Thực sự là mình ko thích loại này lắm, da mình...","thực sự là mình ko thích loại này lắm, da mình...","thực sự là mình không thích loại này lắm, da m...","Actually, I don't like this product very much...."
9,65994,da dầu như mình xài êm.thành phần dịu nhẹ,da dầu như mình xài êm.thành phần dịu nhẹ,da dầu như mình xài êm.thành phần dịu nhẹ,"For oily skin like mine, it's easy to use. Gen..."


In [25]:
# Save into csv
current_datetime = datetime.now().strftime("%Y%m%d_%H%M")
preprocessed_comment_filename = f"preprocessed_comment_{current_datetime}.csv"

mergecomment.to_csv(os.path.join(os.path.dirname(os.getcwd()), "data", "merged", preprocessed_comment_filename), encoding= "utf-8-sig")